# DeepSeek

[DeepSeek](https://www.deepseek.com/) is a Chinese artificial intelligence company that develops LLMs.

In [7]:
pip install torch torchvision pillow


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import copy

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Set image size (use a smaller size if you have limited memory)
imsize = 512 if torch.cuda.is_available() else 128

# Preprocessing: resize, crop, and convert to tensor
loader = transforms.Compose([
    transforms.Resize(imsize),
    transforms.CenterCrop(imsize),
    transforms.ToTensor()])

def image_loader(image_name, size=None):
    """Load image, optionally resize it to a given size (width, height), and convert to a tensor."""
    image = Image.open(image_name)
    if size is not None:
        image = image.resize(size, Image.LANCZOS)
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)
# Load the content image first
content_img = image_loader("/content/Screenshot_2025-02-22_011631-removebg-preview (1).png")  # Your input image

# Convert content image to RGB if it has an alpha channel
if content_img.shape[1] == 4:  # Check if it has 4 channels (RGBA)
    content_img = content_img[:, :3, :, :]  # Keep only the first 3 channels (RGB)

# Get content image size in (height, width) from the tensor shape
content_shape = content_img.shape[2:]  # (height, width)
# Convert to PIL size (width, height)
pil_size = (content_shape[1], content_shape[0])

# Now load the style image and resize it to match the content image size
style_img = image_loader("/content/Gm8d8AObYAImwUr_1743126963833_1743126970437.jpg", size=pil_size)

# Ensure the images are the same size
assert content_img.size() == style_img.size(), "Content and style images must be the same size"

# Function to convert a tensor to a PIL image
unloader = transforms.ToPILImage()
def tensor_to_image(tensor):
    image = tensor.cpu().clone()  # clone the tensor to not modify it
    image = image.squeeze(0)      # remove the batch dimension
    image = unloader(image)
    return image

# Define the Content Loss Module
class ContentLoss(nn.Module):
    def __init__(self, target):
        super(ContentLoss, self).__init__()
        # detach target from the tree
        self.target = target.detach()
        self.loss = 0

    def forward(self, input):
        self.loss = nn.functional.mse_loss(input, self.target)
        return input

# Define the Gram Matrix for Style Loss
def gram_matrix(input):
    a, b, c, d = input.size()  # a=batch size; b=number of feature maps; (c,d)=dimensions
    features = input.view(a * b, c * d)
    G = torch.mm(features, features.t())
    # Normalize the Gram matrix
    return G.div(a * b * c * d)

# Define the Style Loss Module
class StyleLoss(nn.Module):
    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        # Compute and detach the gram matrix target
        self.target = gram_matrix(target_feature).detach()
        self.loss = 0

    def forward(self, input):
        G = gram_matrix(input)
        self.loss = nn.functional.mse_loss(G, self.target)
        return input

# Load a pretrained VGG19 network
cnn = models.vgg19(pretrained=True).features.to(device).eval()

# Normalization module (VGG networks are trained on images normalized in a specific way)
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std  = torch.tensor([0.229, 0.224, 0.225]).to(device)

class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        # Reshape mean and std for normalization
        self.mean = torch.tensor(mean).view(-1, 1, 1)
        self.std  = torch.tensor(std).view(-1, 1, 1)

    def forward(self, img):
        return (img - self.mean) / self.std

# Choose layers to compute style and content losses
content_layers = ['conv_4']
style_layers   = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

# Build the model with losses inserted after chosen layers
def get_style_model_and_losses(cnn, normalization_mean, normalization_std,
                               style_img, content_img):
    cnn = copy.deepcopy(cnn)
    normalization = Normalization(normalization_mean, normalization_std).to(device)
    content_losses = []
    style_losses = []
    # Create a new sequential model
    model = nn.Sequential(normalization)
    i = 0  # Increment every time a convolution layer is added
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = f"conv_{i}"
        elif isinstance(layer, nn.ReLU):
            name = f"relu_{i}"
            # In-place ReLU can modify the computation graph, so use a non in-place version
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = f"pool_{i}"
        elif isinstance(layer, nn.BatchNorm2d):
            name = f"bn_{i}"
        else:
            continue
        model.add_module(name, layer)
        if name in content_layers:
            # Add content loss after this layer
            target = model(content_img).detach()
            content_loss = ContentLoss(target)
            model.add_module(f"content_loss_{i}", content_loss)
            content_losses.append(content_loss)
        if name in style_layers:
            # Add style loss after this layer
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module(f"style_loss_{i}", style_loss)
            style_losses.append(style_loss)
    # Trim the network after the last loss layer
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break
    model = model[:(i + 1)]
    return model, style_losses, content_losses

# Get the model and losses
model, style_losses, content_losses = get_style_model_and_losses(
    cnn, cnn_normalization_mean, cnn_normalization_std, style_img, content_img)

# Create a copy of the content image to optimize; this will be our output image.
input_img = content_img.clone()

# Set optimizer: we optimize the input image in this case.
optimizer = optim.LBFGS([input_img.requires_grad_()])

# Set the weights for the style and content losses
style_weight = 1e6
content_weight = 1

# Run the style transfer
num_steps = 300
print("Optimizing...")
run = [0]
while run[0] <= num_steps:
    def closure():
        # Clamp the input image to [0,1]
        input_img.data.clamp_(0, 1)
        optimizer.zero_grad()
        model(input_img)
        style_score = 0
        content_score = 0
        for sl in style_losses:
            style_score += sl.loss
        for cl in content_losses:
            content_score += cl.loss
        loss = style_weight * style_score + content_weight * content_score
        loss.backward()
        run[0] += 1
        if run[0] % 50 == 0:
            print(f"Step {run[0]}: Style Loss: {style_score.item():4f} Content Loss: {content_score.item():4f}")
        return loss
    optimizer.step(closure)

# Clamp the final output and save the image
input_img.data.clamp_(0, 1)
output = tensor_to_image(input_img)
output.save("output_ghibli.jpg")
print("Output image saved as output_ghibli.jpg")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:02<00:00, 216MB/s]
<ipython-input-9-4a2f41f94cd8>:97: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tor

Optimizing...
Step 50: Style Loss: 0.000441 Content Loss: 55.372227
Step 100: Style Loss: 0.000099 Content Loss: 52.588104
Step 150: Style Loss: 0.000049 Content Loss: 49.079247


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import copy

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Set image size (use a smaller size if you have limited memory)
imsize = 512 if torch.cuda.is_available() else 128

# Preprocessing: resize, crop, and convert to tensor
loader = transforms.Compose([
    transforms.Resize(imsize),
    transforms.CenterCrop(imsize),
    transforms.ToTensor()])

def image_loader(image_name):
    """Load image and convert to a tensor"""
    image = Image.open(image_name)
    # Add a batch dimension and convert to tensor
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)

# Load content and style images (ensure they are the same size)
content_img = image_loader("/content/Screenshot_2025-02-22_011631-removebg-preview (1).png")  # Your input image
style_img = image_loader("/content/Gm8d8AObYAImwUr_1743126963833_1743126970437.jpg")     # A sample Ghibli-style image

assert content_img.size() == style_img.size(), "Content and style images must be the same size"

# Function to convert a tensor to a PIL image
unloader = transforms.ToPILImage()
def tensor_to_image(tensor):
    image = tensor.cpu().clone()  # clone the tensor to not modify it
    image = image.squeeze(0)      # remove the batch dimension
    image = unloader(image)
    return image

# Define the Content Loss Module
class ContentLoss(nn.Module):
    def __init__(self, target):
        super(ContentLoss, self).__init__()
        # detach target from the tree
        self.target = target.detach()
        self.loss = 0

    def forward(self, input):
        self.loss = nn.functional.mse_loss(input, self.target)
        return input

# Define the Gram Matrix for Style Loss
def gram_matrix(input):
    a, b, c, d = input.size()  # a=batch size; b=number of feature maps; (c,d)=dimensions
    features = input.view(a * b, c * d)
    G = torch.mm(features, features.t())
    # Normalize the Gram matrix
    return G.div(a * b * c * d)

# Define the Style Loss Module
class StyleLoss(nn.Module):
    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        # Compute and detach the gram matrix target
        self.target = gram_matrix(target_feature).detach()
        self.loss = 0

    def forward(self, input):
        G = gram_matrix(input)
        self.loss = nn.functional.mse_loss(G, self.target)
        return input

# Load a pretrained VGG19 network
cnn = models.vgg19(pretrained=True).features.to(device).eval()

# Normalization module (VGG networks are trained on images normalized in a specific way)
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std  = torch.tensor([0.229, 0.224, 0.225]).to(device)

class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        # Reshape mean and std for normalization
        self.mean = torch.tensor(mean).view(-1, 1, 1)
        self.std  = torch.tensor(std).view(-1, 1, 1)

    def forward(self, img):
        return (img - self.mean) / self.std

# Choose layers to compute style and content losses
content_layers = ['conv_4']
style_layers   = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

# Build the model with losses inserted after chosen layers
def get_style_model_and_losses(cnn, normalization_mean, normalization_std,
                               style_img, content_img):
    cnn = copy.deepcopy(cnn)

    normalization = Normalization(normalization_mean, normalization_std).to(device)

    content_losses = []
    style_losses = []

    # Create a new sequential model
    model = nn.Sequential(normalization)

    i = 0  # increment every time a convolution layer is added
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = f"conv_{i}"
        elif isinstance(layer, nn.ReLU):
            name = f"relu_{i}"
            # In-place ReLU can modify the computation graph, so use a non in-place version
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = f"pool_{i}"
        elif isinstance(layer, nn.BatchNorm2d):
            name = f"bn_{i}"
        else:
            continue

        model.add_module(name, layer)

        if name in content_layers:
            # Add content loss after this layer
            target = model(content_img).detach()
            content_loss = ContentLoss(target)
            model.add_module(f"content_loss_{i}", content_loss)
            content_losses.append(content_loss)

        if name in style_layers:
            # Add style loss after this layer
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module(f"style_loss_{i}", style_loss)
            style_losses.append(style_loss)

    # Trim the network after the last loss layer
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break
    model = model[:(i + 1)]

    return model, style_losses, content_losses

# Get the model and losses
model, style_losses, content_losses = get_style_model_and_losses(
    cnn, cnn_normalization_mean, cnn_normalization_std, style_img, content_img)

# Create a copy of the content image to optimize; this will be our output image.
input_img = content_img.clone()

# Set optimizer: we optimize the input image in this case.
optimizer = optim.LBFGS([input_img.requires_grad_()])

# Set the weights for the style and content losses
style_weight = 1e6
content_weight = 1

# Run the style transfer
num_steps = 300
print("Optimizing...")
run = [0]
while run[0] <= num_steps:
    def closure():
        # Clamp the input image to [0,1]
        input_img.data.clamp_(0, 1)

        optimizer.zero_grad()
        model(input_img)
        style_score = 0
        content_score = 0

        for sl in style_losses:
            style_score += sl.loss
        for cl in content_losses:
            content_score += cl.loss

        loss = style_weight * style_score + content_weight * content_score
        loss.backward()

        run[0] += 1
        if run[0] % 50 == 0:
            print(f"Step {run[0]}: Style Loss: {style_score.item():4f} Content Loss: {content_score.item():4f}")
        return loss

    optimizer.step(closure)

# Clamp the final output
input_img.data.clamp_(0, 1)
# Save the output image
output = tensor_to_image(input_img)
output.save("output_ghibli.jpg")
print("Output image saved as output_ghibli.jpg")


AssertionError: Content and style images must be the same size